# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 13 2022 8:56AM,252947,16,SHL-8896969,"SHL Pharma, LLC",Released
1,Dec 13 2022 8:54AM,252946,12,U1147,"Uniderm USA, Inc",Released
2,Dec 13 2022 8:52AM,252945,12,HH-38737,Hush Hush,Released
3,Dec 13 2022 8:52AM,252945,12,HH-38736,Hush Hush,Released
4,Dec 13 2022 8:36AM,252943,19,TR-RN-12142022,"GCH Granules USA, Inc.",Released
5,Dec 13 2022 8:34AM,252942,20,8896940,"Exact-Rx, Inc.",Released
6,Dec 13 2022 8:34AM,252942,20,8896941,"Exact-Rx, Inc.",Released
7,Dec 13 2022 8:34AM,252942,20,8896942,"Exact-Rx, Inc.",Released
8,Dec 13 2022 8:34AM,252942,20,8896943,"Exact-Rx, Inc.",Released
9,Dec 13 2022 8:34AM,252942,20,8896944,"Exact-Rx, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
29,252942,Released,5
30,252943,Released,1
31,252945,Released,2
32,252946,Released,1
33,252947,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
252942,NaN,NaN,5.0
252943,NaN,NaN,1.0
252945,NaN,NaN,2.0
252946,NaN,NaN,1.0
252947,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
252812,6.0,0.0,12.0
252813,0.0,0.0,1.0
252818,0.0,0.0,3.0
252823,0.0,1.0,0.0
252832,0.0,1.0,4.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
252812,6,0,12
252813,0,0,1
252818,0,0,3
252823,0,1,0
252832,0,1,4


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,252812,6,0,12
1,252813,0,0,1
2,252818,0,0,3
3,252823,0,1,0
4,252832,0,1,4


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,252812,6,,12
1,252813,,,1
2,252818,,,3
3,252823,,1,
4,252832,,1,4


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 13 2022 8:56AM,252947,16,"SHL Pharma, LLC"
1,Dec 13 2022 8:54AM,252946,12,"Uniderm USA, Inc"
2,Dec 13 2022 8:52AM,252945,12,Hush Hush
4,Dec 13 2022 8:36AM,252943,19,"GCH Granules USA, Inc."
5,Dec 13 2022 8:34AM,252942,20,"Exact-Rx, Inc."
10,Dec 13 2022 7:26AM,252937,19,"AdvaGen Pharma, Ltd"
11,Dec 12 2022 7:31PM,252936,21,"NBTY Global, Inc."
12,Dec 12 2022 5:46PM,252934,15,"Virtus Pharmaceuticals, LLC"
32,Dec 12 2022 5:42PM,252933,15,"Brookfield Pharmaceuticals, LLC"
95,Dec 12 2022 5:20PM,252932,15,"Alliance Pharma, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Dec 13 2022 8:56AM,252947,16,"SHL Pharma, LLC",,,1
1,Dec 13 2022 8:54AM,252946,12,"Uniderm USA, Inc",,,1
2,Dec 13 2022 8:52AM,252945,12,Hush Hush,,,2
3,Dec 13 2022 8:36AM,252943,19,"GCH Granules USA, Inc.",,,1
4,Dec 13 2022 8:34AM,252942,20,"Exact-Rx, Inc.",,,5
5,Dec 13 2022 7:26AM,252937,19,"AdvaGen Pharma, Ltd",,,1
6,Dec 12 2022 7:31PM,252936,21,"NBTY Global, Inc.",,,1
7,Dec 12 2022 5:46PM,252934,15,"Virtus Pharmaceuticals, LLC",,,20
8,Dec 12 2022 5:42PM,252933,15,"Brookfield Pharmaceuticals, LLC",,,63
9,Dec 12 2022 5:20PM,252932,15,"Alliance Pharma, Inc.",,,80


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 13 2022 8:56AM,252947,16,"SHL Pharma, LLC",1,,
1,Dec 13 2022 8:54AM,252946,12,"Uniderm USA, Inc",1,,
2,Dec 13 2022 8:52AM,252945,12,Hush Hush,2,,
3,Dec 13 2022 8:36AM,252943,19,"GCH Granules USA, Inc.",1,,
4,Dec 13 2022 8:34AM,252942,20,"Exact-Rx, Inc.",5,,
5,Dec 13 2022 7:26AM,252937,19,"AdvaGen Pharma, Ltd",1,,
6,Dec 12 2022 7:31PM,252936,21,"NBTY Global, Inc.",1,,
7,Dec 12 2022 5:46PM,252934,15,"Virtus Pharmaceuticals, LLC",20,,
8,Dec 12 2022 5:42PM,252933,15,"Brookfield Pharmaceuticals, LLC",63,,
9,Dec 12 2022 5:20PM,252932,15,"Alliance Pharma, Inc.",80,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 13 2022 8:56AM,252947,16,"SHL Pharma, LLC",1,,
1,Dec 13 2022 8:54AM,252946,12,"Uniderm USA, Inc",1,,
2,Dec 13 2022 8:52AM,252945,12,Hush Hush,2,,
3,Dec 13 2022 8:36AM,252943,19,"GCH Granules USA, Inc.",1,,
4,Dec 13 2022 8:34AM,252942,20,"Exact-Rx, Inc.",5,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 13 2022 8:56AM,252947,16,"SHL Pharma, LLC",1.0,NaN,NaN
1,Dec 13 2022 8:54AM,252946,12,"Uniderm USA, Inc",1.0,NaN,NaN
2,Dec 13 2022 8:52AM,252945,12,Hush Hush,2.0,NaN,NaN
3,Dec 13 2022 8:36AM,252943,19,"GCH Granules USA, Inc.",1.0,NaN,NaN
4,Dec 13 2022 8:34AM,252942,20,"Exact-Rx, Inc.",5.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 13 2022 8:56AM,252947,16,"SHL Pharma, LLC",1.0,0.0,0.0
1,Dec 13 2022 8:54AM,252946,12,"Uniderm USA, Inc",1.0,0.0,0.0
2,Dec 13 2022 8:52AM,252945,12,Hush Hush,2.0,0.0,0.0
3,Dec 13 2022 8:36AM,252943,19,"GCH Granules USA, Inc.",1.0,0.0,0.0
4,Dec 13 2022 8:34AM,252942,20,"Exact-Rx, Inc.",5.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2781601,70.0,2.0,6.0
12,758819,4.0,0.0,0.0
15,758799,163.0,0.0,0.0
16,505846,2.0,0.0,0.0
19,1517359,9.0,13.0,0.0
20,505804,10.0,18.0,0.0
21,505759,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2781601,70.0,2.0,6.0
1,12,758819,4.0,0.0,0.0
2,15,758799,163.0,0.0,0.0
3,16,505846,2.0,0.0,0.0
4,19,1517359,9.0,13.0,0.0
5,20,505804,10.0,18.0,0.0
6,21,505759,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,70.0,2.0,6.0
1,12,4.0,0.0,0.0
2,15,163.0,0.0,0.0
3,16,2.0,0.0,0.0
4,19,9.0,13.0,0.0
5,20,10.0,18.0,0.0
6,21,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,70.0
1,12,Released,4.0
2,15,Released,163.0
3,16,Released,2.0
4,19,Released,9.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,6.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,2.0,0.0,0.0,0.0,13.0,18.0,1.0
Released,70.0,4.0,163.0,2.0,9.0,10.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,6.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,2.0,0.0,0.0,0.0,13.0,18.0,1.0
2,Released,70.0,4.0,163.0,2.0,9.0,10.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,6.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,2.0,0.0,0.0,0.0,13.0,18.0,1.0
2,Released,70.0,4.0,163.0,2.0,9.0,10.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()